In [1]:
import torch
import torch.nn as nn
import numpy as np
import wandb
import math

c:\Users\Leonard\anaconda3\envs\pytorch_nn_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project="nn-assignment-1-pytorch-ooo", group="epochs=100, batch_size=25, lr=0.001, network=ReLU")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leonard-puskac (fiit-nn-2023-lp-vs). Use `wandb login --relogin` to force relogin


## Opening and pre-processing the data

In [3]:
with open("data/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


Data normalization, dropping columns with correlation lower than 0.1 with the resulting class, and spliting into train and validation sets

In [4]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        #x[x<0] *= -1
        #x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)

In [5]:
def train_test_split_2(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.7)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = input_data[train_idx,:], input_data[test_idx,:]
    train_labels, test_labels = input_labels[train_idx], input_labels[test_idx]
    return train_data, test_data, train_labels, test_labels

In [6]:
x_norm = normalise_2darray(all_data)

corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [labels[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)

x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape


(208, 37)

In [7]:
#from sklearn.model_selection import train_test_split
#x_train, x_val = train_test_split(x_norm, test_size=0.3)
#y_train, y_val = train_test_split(labels, test_size=0.3)
x_train, x_val, y_train, y_val = train_test_split_2(x_norm, labels)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)


(145, 37) (63, 37) (145,) (63,)


Setting up "device", converting the numpy arrays into torch tensors and passing them to device 

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device).reshape(-1,1).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device).reshape(-1,1).to(device)

print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)


torch.Size([145, 37]) torch.Size([63, 37]) torch.Size([145, 1]) torch.Size([63, 1])


In [10]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network_old(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(128, 128),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(128, 128),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(128, 64),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(64, 64),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(64, 64),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(64, 64),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Linear(64, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, X, y, X_val, y_val, n_epochs, loss_fn, optimizer, batch_size=30):
        batches = torch.arange(0, len(X), batch_size)
        for epoch in range(n_epochs):
            best_loss = 20000
            best_acc = 0
            self.train()
            for batch_start in batches:
                X_batch = X[batch_start:(batch_start+batch_size)]
                y_batch = y[batch_start:(batch_start+batch_size)]

                prediction = self.forward(X_batch)
                loss = loss_fn(prediction, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                acc = (prediction.round() == y_batch).float().mean()
                #wandb.log({"loss": loss})
                #wandb.log({"acc": acc})
                best_acc = acc if acc > best_acc else best_acc
                best_loss = loss if loss < best_loss else best_loss
            
            #wandb.log({"acc": best_acc})
            
            self.eval()
            y_pred = self.network(X_val)
            val_acc = (y_pred.round() == y_val).float().mean()
            val_loss = loss_fn(y_pred, y_val)
            #wandb.log({"val_acc": val_acc})
            wandb.log({"loss": best_loss, "acc": best_acc, "val_acc": val_acc, "val_loss": val_loss})
            print(f"Epoch {epoch}:\ttrain_loss: {best_loss:.4f}\ttrain_acc: {best_acc:.4f}\tval_loss: {val_loss:.4f}\tval_acc: {val_acc:.4f}")
                
    


In [11]:

model = Network(input_size=x_train.shape[1], output_size=1).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.network.parameters(), lr=0.001, betas=(0.9, 0.99))
model.train()
wandb.watch(model)
model.fit(X=x_train, y=y_train, X_val=x_val, y_val=y_val, n_epochs=100, loss_fn=loss_fn, optimizer=optimizer, batch_size=25)

Epoch 0:	train_loss: 0.6818	train_acc: 0.7200	val_loss: 0.6913	val_acc: 0.5079
Epoch 1:	train_loss: 0.6658	train_acc: 0.7200	val_loss: 0.6835	val_acc: 0.5873
Epoch 2:	train_loss: 0.6061	train_acc: 0.7600	val_loss: 0.6608	val_acc: 0.6032
Epoch 3:	train_loss: 0.4319	train_acc: 0.9000	val_loss: 0.6501	val_acc: 0.6349
Epoch 4:	train_loss: 0.3197	train_acc: 0.9000	val_loss: 0.6243	val_acc: 0.6825
Epoch 5:	train_loss: 0.2904	train_acc: 0.8800	val_loss: 0.5312	val_acc: 0.7937
Epoch 6:	train_loss: 0.2781	train_acc: 0.9000	val_loss: 0.5119	val_acc: 0.7143
Epoch 7:	train_loss: 0.2435	train_acc: 0.9200	val_loss: 0.5576	val_acc: 0.7778
Epoch 8:	train_loss: 0.2052	train_acc: 0.9200	val_loss: 0.5804	val_acc: 0.7937
Epoch 9:	train_loss: 0.2137	train_acc: 0.9200	val_loss: 0.5968	val_acc: 0.7619
Epoch 10:	train_loss: 0.1909	train_acc: 0.9200	val_loss: 0.6344	val_acc: 0.7619
Epoch 11:	train_loss: 0.1826	train_acc: 0.9500	val_loss: 0.6639	val_acc: 0.7460
Epoch 12:	train_loss: 0.1735	train_acc: 0.9500	val